In [5]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import pickle

This notebook is supposed to demonstrate the application of PCA.
As a dataset we will use the MNIST database of handwritten digits. The dataset can be found under the follwing link:

http://yann.lecun.com/exdb/mnist/

But instead of manually downloading the dataset, we can also just use the function 'fetch_openml' from 'sklearn.datasets':

In [9]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [16]:
x.shape, y.shape

((70000, 784), (70000,))

First we need to split the dataset into a train and a test partition. We will use the train dataset to build the model and then use the test set to verify this model. The computation time for the model will depend on the train set size. So please set the size according to your machine's capabilities.

(Especially the SVD might take some time.)

In [18]:
# n_train_samples = 1000
# n_train_samples = 2000
# n_train_samples = 4000
n_train_samples = 10_000
# n_train_samples = 20_000
# n_train_samples = 50_000

x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, train_size=n_train_samples)
#x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, train_size=0.65)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

NameError: name 'x_train' is not defined

Each sample is represented by a vector of size 28*28=784. Each of these 784 numbers represents the gray-scale value of a pixel in a range from 0 to 255 (8 bit).

Let's look at sample:

In [ ]:
plt.matshow(x_train[1].reshape(28,28), cmap="gray")
plt.colorbar()

In [ ]:
x_train.min(), x_train.max()

In [ ]:
y_train[1]

For the PCA, we first need to 'center' the data by calculating the mean data vector and then subtract this vector from each sample:

In [ ]:
x_train_mean = x_train.mean(axis=0)
x_train_mean.shape

In [ ]:
# this is an image of the mean data vector
plt.matshow(x_train_mean.reshape(28,28), cmap="gray", vmin=0, vmax=255)
plt.colorbar()

In [ ]:
x_train_centered = x_train - x_train_mean

Now we can execute the main part of the PCA - the matrix decomposition. There are two equivalent ways to do this. We can either use SVD on the data matrix or use Eigenvalue decomposition on the data covariance matrix.

In [ ]:
U, s, Vt = np.linalg.svd(x_train_centered/np.sqrt(len(x_train)))

In [ ]:
U.shape, Vt.shape

In [ ]:
lambda_svd = s**2
modes_svd = Vt

We can plot the 'energy' of the modes compared to the total energy. By finding the intercept with the value of the 'energy criterion' (e.g. 0.95 of the total energy) we can figure out, how many modes we need to faithfully represent our data.

In [ ]:
plt.plot(np.cumsum(lambda_svd)/np.sum(lambda_svd))
plt.axhline(0.95, ls=":")

In [ ]:
lambda_svd_sum = np.sum(lambda_svd)
lambda_svd_cumsum = np.cumsum(lambda_svd)
for i in range(20):
    print(i, lambda_svd[i], lambda_svd_cumsum[i]/lambda_svd_sum)

The same results can be achieved by using Eigenvalue decomposition:

In [ ]:
C = x_train_centered.T @ x_train_centered / len(x_train_centered)

In [ ]:
lambda_eig, modes_eig = np.linalg.eig(C)
lambda_eig = lambda_eig.astype(np.float64)
modes_eig = modes_eig.T.astype(np.float64)

In [ ]:
plt.plot(np.cumsum(lambda_eig)/np.sum(lambda_eig))
plt.axhline(0.95, ls=":")

Let's check if the modes are equivalent:

In [ ]:
plt.matshow(modes_svd[0].reshape(28,28))
plt.colorbar()

In [ ]:
plt.matshow(modes_eig[0].reshape(28,28))
plt.colorbar()


In [ ]:
for i in range(10):
    plt.matshow(modes_svd[i].astype("double").reshape(28,28))
    plt.colorbar()
    plt.gca().set_title(f"Mode {i} SVD")
    plt.show()

    plt.matshow(modes_eig[i].astype("double").reshape(28,28))
    plt.colorbar()
    plt.gca().set_title(f"Mode {i} Eig")
    plt.show()

(Except for the +/- sign, these images should be equal for SVD and Eigenvalue decomposition.)

Now we can calculate the number of modes we need to represent the samples according to the energy criterion (e.g. 0.95):

In [ ]:
# These two methods are equivalent:
# (a)
r_truncation = np.where((np.cumsum(lambda_eig)/np.sum(lambda_eig)) >= 0.95)[0][0]
print(r_truncation)

# (b)
for i in range(784):
    if np.sum(lambda_eig[:i+1]) / np.sum(lambda_eig) >= 0.95:
        r_truncation = i
        break

r_truncation

Let's see how well the compression works:

In [ ]:
plt.matshow(x_test[0].reshape(28,28))
plt.colorbar()

We can compress a data vector by projecting it into a lower dimensional space. We can do this by simply by computing the product with the (truncated) matrix of the modes. To restore the original vector, we then just have to repeat the multiplication and add the mean vector:

In [ ]:
r = r_truncation
x_compressed = x_test[0] @ Vt[:r].T
x_restored = x_compressed @ Vt[:r] + x_train_mean
x_compressed.shape, x_restored.shape

In [ ]:
plt.matshow(x_restored.reshape(28,28))
plt.colorbar()

Looking at the result, we can see that the restored version of the data vector is not a perfect match with the original vector. Some information has been lost. But it should still be a pretty good approximation of the original data.

Let's check how well the lower dimensional representation works with different truncation ranks:

In [ ]:
i = 0
plt.matshow(x_test[i].reshape(28,28))
plt.colorbar()
plt.show()

for r in [0, 5, 10, 20, 50, 100, 200, 500, 784]:
    #fig, ax = plt.subplots(2, figsize=(10,4))
    x_rec = x_test[i] @ modes_svd[:r].T @ modes_svd[:r] + x_train_mean
    #x_rec = np.clip(x_rec, 0., 255.)
    #plt.matshow(x_rec.reshape(28,28), vmin=0, vmax=255)
    plt.matshow(x_rec.reshape(28,28))
    #plt.matshow(x_rec.reshape(28,28), cmap="gray", vmin=0, vmax=255)
    plt.colorbar()
    plt.gca().set_title(f"Rank {r} Approximation")
    plt.show()

    print(r, np.sum(lambda_svd[:r])/np.sum(lambda_svd))

The lower dimensional projection of the data can be used in other downstream tasks. As an example we will now build a very simple Classifier to recognize the handwritten digits. Using the projected data can make downstream tasks more efficient, because of the lower dimensionality of the data.

(This demonstration is not related to your assignment)

In [14]:
from sklearn.linear_model import SGDClassifier

In [15]:
r = 20

x_train_compressed = x_train @ modes_svd[:r].T
x_test_compressed = x_test @ modes_svd[:r].T

reg = SGDClassifier().fit(x_train_compressed, y_train)
y_test_pred = reg.predict(x_test_compressed)

NameError: name 'x_train' is not defined

In [ ]:
x_train_compressed.shape, x_test_compressed.shape

In [ ]:
# how many train samples are correctly classified?
reg.score(x_train_compressed, y_train)

In [ ]:
# how many test samples are correctly classified?
reg.score(x_test_compressed, y_test)

In [ ]:
print(reg.predict(x_test_compressed[:10]))
print([y for y in y_test[:10]])

If you are still having troubles understanding PCA, the following lectures by Brunton & Kutz might be helpful for you:

http://www.databookuw.com/page-2/page-4/

You can also check the follwing paper for applications of PCA/POD in aerospace engineering:

http://dx.doi.org/10.14279/depositonce-8512